## Combining tracing rabbit and progress per step.

As the model `tracing rabbit` and `progress per step` work well, and one of them in charge of steering and the other in charge of speed, it is a good approach to combining two of them.

We will introduce the formal way to combine two reward functions in this session.

Generally, while trying to combine two reward functions, we get clear understanding of both functions and then implement the program logic into one function: 

In [1]:
import math

def reward_function(params):
    
    # Read all input parameters
    all_wheels_on_track = params['all_wheels_on_track']
    x = params['x']
    y = params['y']
    distance_from_center = params['distance_from_center']
    is_left_of_center = params['is_left_of_center']
    heading = params['heading']
    progress = params['progress']
    steps = params['steps']
    speed = params['speed']
    steering_angle = params['steering_angle']
    track_width = params['track_width']
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    is_offtrack = params['is_offtrack']
    
    ###### expected progress 
    reward = 0.0
    
    expected_steps = 105 # for re:invent 2018, finish one lap in 7 seconds, 15*7 = 105 steps
    
    expected_progress_per_step = 100/expected_steps
    
    expected_progress = expected_progress_per_step * steps
    
    progress_reward = progress/expected_progress
    
    ##### tracing rabbit
    waypoints_length = len(waypoints)
    
    front_waypoint = params['closest_waypoints'][1]
    
    rabbit_waypoint = front_waypoint + 1
    
    if (rabbit_waypoint >= waypoints_length):
        rabbit_waypoint = rabbit_waypoint % waypoints_length
    
    rabbit = [waypoints[rabbit_waypoint][0],waypoints[rabbit_waypoint][1]]
    
    radius = math.hypot(x - rabbit[0], y - rabbit[1])
    
    pointing = [0,0]
    pointing[0] = x + (radius * math.cos(heading))
    pointing[1] = y + (radius * math.sin(heading))
    
    vector_delta = math.hypot(pointing[0] - rabbit[0], pointing[1] - rabbit[1])
    
    
    
    rabbit_reward = ( 1 - ( vector_delta / (radius * 2)))
    
    reward += progress_reward
    reward += rabbit_reward
    
    
#     print(params)
#     print("=====progress reward: %f" % progress_reward)
#     print("=====rabbit reward: %f" % rabbit_reward)
#     print("=====reward: %f" % reward)
    
    return float(reward)

In [4]:
from race_utils import SampleGenerator
generator = SampleGenerator()
params = generator.random_sample()

In [5]:
reward_function(params)

1.2053483223866839

## Configs and result


The following chart is the training metrics of the `Progress per step and tracing rabbit` model with higer speed. 

In this model, we set the action space to discrete space.

One benifit of setting discrete action space is tuning different speed for different steering angle.

We want the car to keep in 1.4 meters per second while taking the U turn, at the same time we hope the car to go fater while it is runing on straight race line.

So we set the speed of 30 degrees and -30 degress to 1.4 meters per second. And we set the min speed of 0 degree to 2 meters per second. 



<img src="./images/09_result.png" width = "300"  alt="result"  />



And the evaluation results are listed below.

All of the three evalutions reach 100% progress, with the time used being 10 seconds.

And it took 31 seconds to finish 3 laps.

<img src="./images/09_evaluation.png" width = "300"  alt="evaluation"  />


Action space and other hyper parameters are listed as below:

<img src="./images/09_action_space.png" width = "300"  alt="action space"  />

<img src="./images/09_hyper_parameters.png" width = "300"  alt="hyper parameters"  />

